# Menginstall Dependency

In [ ]:
!pip install "pyarrow==19.0.0" --force-reinstall -q
!pip install -U datasets bitsandbytes sentencepiece -q
!pip install -U "transformers>=4.44.0" "accelerate>=0.34.0" "peft>=0.11.0" -q
!pip install python-dotenv

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.0 which is incompatible.


# Import Library

In [ ]:
import os
import glob
import torch
from google.colab import drive
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model

# Mount Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)
LOCAL_DATA_DIR = "/content/drive/MyDrive/GColab_AITFIndonesia/Tim1_dataset_cpt"

Mounted at /content/drive


# Konfigurasi Global & Pemuatan Data

In [ ]:
MODEL_NAME = "Qwen/Qwen3-8B"

all_files = glob.glob(os.path.join(LOCAL_DATA_DIR, "*.txt"))
print(f"Total file.txt yang ditemukan: {len(all_files)}")

if not all_files:
    raise ValueError(f"Tidak ada file.txt yang ditemukan di {LOCAL_DATA_DIR}. Periksa path dan proses unzip.")

raw_dataset = load_dataset("text", data_files=all_files, split="train")

print(f"Jumlah baris sebelum cleaning: {len(raw_dataset)}")
raw_dataset = raw_dataset.filter(lambda x: x['text'] is not None and x['text'].strip() != '')
print(f"Jumlah baris setelah cleaning: {len(raw_dataset)}")

Total file.txt yang ditemukan: 1
Jumlah baris sebelum cleaning: 139105
Jumlah baris setelah cleaning: 139105


In [ ]:
raw_dataset['text'][0]

'Di tengah beragam kecaman terhadap otoritas Zionis, media Israel, the Jerusalem Post menulis artikel yang memuji kurikulum pembelajaran Indonesia. The Jerusalem Post mengutip sebuah penilaian baru oleh IMPACT-se, sebuah lembaga penelitian dan kebijakan internasional, yang menyoroti pertumbuhan inklusivitas dalam \'Kurikulum Merdeka\' di Indonesia. Laporan itu menilai bahwa buku teks nasional Indonesia sedang mengalami transformasi yang signifikan dalam perbaikan sikap terhadap Yahudi, Israel, dan kelompok minoritas. Buku teks ini sekaligus mempromosikan kesetaraan gender dan membingkai ulang konsep-konsep seperti jihad. Studi menganalisis lebih dari empat puluh buku teks humaniora dari kurikulum baru dan membandingkannya dengan edisi sebelumnya. IMPACT-se menemukan keselarasan yang lebih erat dengan standar pendidikan perdamaian dan toleransi yang diturunkan dari UNESCO. "Sangat menggembirakan melihat buku teks Indonesia berada di jalur yang tegas menuju pertumbuhan inklusivitas," uja

# Split Dataset

In [ ]:
print("Memisahkan dataset 90:10...")
split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})
print(dataset)

Memisahkan dataset 90:10...
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 125194
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 13911
    })
})


In [ ]:
for i in range(3):
  display(
      DatasetDict({
    'train': dataset['train']['text'][i]
    }))

DatasetDict({
    train: Dalam tradisi perkawinan masyarakat sumba khususnya sumba barat daya.Antara pihak laki-laki dan wanita sesuai adat perkawinan di sumba barat daya.Ada beberapa tahap yang harus dilakukan yaitu; Masuk minta (Melamar) Masuk minta (Melamar) Perkawinan dan harga seorang perempuan Perkawinan bagi orang sumba, khususnya di suku wewena sumba barat daya,dalam artiter tentu bisa saya katakan sebagai suatu upaya resiprositas antar suku,antara si pemberi perempuan dan si penerima atau "pembeli" perempuan. Kebiasaan yang semacam iniagak umum banyak tempat yang berbudaya pariakal atau patriarkal. Dalam konteks orang sumba suku wewena, dikenal beberapa macam jenis perkawinan yang perlu di ketahui sebelum saya membuat analisis kritis dengan teori resipiositas. Perkawinan "Normal"(cara yang dianjurkan). Perkawinan dan harga seorang perempuan Perkawinan bagi orang sumba, khususnya di suku wewena sumba barat daya,dalam artiter tentu bisa saya katakan sebagai suatu upaya resiprosi

DatasetDict({
    train: Dalam kehidupan sehari-hari, kita melakukan pengukuran untuk berbagai keperluan, mulai dari perdagangan, industri, kesehatan, transportasi, hingga pengembangan sains. Untuk mendapatkan hasil pengukuran yang dapat diterima oleh semua orang di dunia, dibutuhkan sistem pengukuran yang diakui secara internasional. Metrologi merupakan disiplin ilmu yang mempelajari cara-cara pengukuran, kalibrasi dan akurasi di bidang industri, ilmu pengetahuan dan teknologi. Kegiatan metrologi yang berbasis pada satuan ukuran dalam Sistem Internasional Satuan atau The International System of Units (SI). Prabowo-Sandi Tolak Hasil Pemilu 2019 dan Minta Pendukung Jaga Ketertiban Saat Demokrat dan PAN Dikabarkan Merapat ke Kubu Jokowi, Cak Imin: Saya Kira Cukup SI merupakan buah dari Konvensi Meter, yaitu kesepakatan antar negara pada 20 Mei 1875 untuk menciptakan suatu sistem pengukuran yang konsisten. SI menetapkan definisi untuk tujuh satuan dasar, yaitu sekon, meter, kilogram, ampe

DatasetDict({
    train: Komjen Pol (Purn) Suyitno Landung adalah mantan Kepala Badan Reserse dan Kriminal Polri pada tahun 2004-2005. Pada tanggal 13 Desember 2005, ia ditetapkan sebagai tersangka penyalahgunaan wewenang pada saat menangani kasus pembobolan Bank BNI dengan tersangka Adrian Waworuntu. Ia divonis satu tahun enam bulan penjara oleh Pengadilan Negeri Jakarta Selatan pada Oktober 2006 dan ditahan Markas Besar Polri. Menurut rencana seharusnya ia kemudian dipindahkan ke LP Cipinang, tetapi pada Januari 2007 Landung dipindah ke tahanan Markas Komando Brigade Mobil di Depok, Jawa Barat karena "dikhawatirkan akan dianiaya" tahanan di Cipinang. Referensi Tokoh Polri Tokoh yang tidak memiliki informasi tahun kelahiran Koruptor Indonesia<|im_end|>
})

# Tokenisasi dan Chunking

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
block_size = 1024

def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}

    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size

    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }

    result["labels"] = result["input_ids"].copy()
    return result

print("Melakukan Tokenisasi Awal...")
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names
)

print(f"Melakukan Packing (Concatenation) ke {block_size} token...")
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
)

print(f"Dataset Final (Packed): {lm_datasets}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Melakukan Tokenisasi Awal...
Melakukan Packing (Concatenation) ke 1024 token...
Dataset Final (Packed): DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 185427
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20500
    })
})


# Visualisasi Data Untuk Training

In [ ]:
for i in range(5):
  print('Block size     : ',len(lm_datasets['train']['input_ids'][i]))
  print('Input ids      : ',lm_datasets['train']['input_ids'][i])
  print('Attention Mask : ',lm_datasets['train']['attention_mask'][i])
  print('Labels         : ',lm_datasets['train']['labels'][i])
  print('Contoh teks:')
  display(tokenizer.decode(lm_datasets['train'][i]['input_ids'], skip_special_tokens=False))

  print('\n')

Block size     :  1024
Input ids      :  [35, 16823, 4685, 13229, 65093, 672, 73176, 296, 91280, 2629, 4645, 595, 12800, 355, 22891, 2629, 4645, 3619, 266, 1899, 64, 875, 406, 5059, 281, 6996, 585, 326, 14624, 2852, 14624, 9101, 77753, 6255, 15537, 84695, 993, 266, 65093, 672, 73176, 1853, 2629, 4645, 3619, 266, 1899, 64, 875, 3235, 84938, 90093, 391, 10371, 58497, 18840, 49995, 379, 1315, 84, 26, 19868, 3101, 28337, 64, 320, 39149, 15554, 8, 19868, 3101, 28337, 64, 320, 39149, 15554, 8, 3616, 74, 672, 73176, 9101, 71348, 511, 84168, 817, 3262, 10386, 3616, 74, 672, 73176, 85650, 49607, 2629, 4645, 11, 595, 12800, 355, 22891, 1853, 913, 12133, 289, 365, 7157, 2629, 4645, 3619, 266, 1899, 64, 11991, 16823, 1947, 2015, 15632, 84, 39988, 53937, 61688, 8656, 58622, 913, 35309, 705, 12589, 592, 573, 3630, 21226, 3196, 277, 913, 12133, 11, 517, 5059, 4403, 281, 1377, 72, 817, 3262, 10386, 9101, 4403, 281, 92682, 38372, 330, 79, 8858, 12294, 1, 817, 3262, 10386, 13, 730, 3065, 3473, 41702, 10

'Dalam tradisi perkawinan masyarakat sumba khususnya sumba barat daya.Antara pihak laki-laki dan wanita sesuai adat perkawinan di sumba barat daya.Ada beberapa tahap yang harus dilakukan yaitu; Masuk minta (Melamar) Masuk minta (Melamar) Perkawinan dan harga seorang perempuan Perkawinan bagi orang sumba, khususnya di suku wewena sumba barat daya,dalam artiter tentu bisa saya katakan sebagai suatu upaya resiprositas antar suku,antara si pemberi perempuan dan si penerima atau "pembeli" perempuan. Kebiasaan yang semacam iniagak umum banyak tempat yang berbudaya pariakal atau patriarkal. Dalam konteks orang sumba suku wewena, dikenal beberapa macam jenis perkawinan yang perlu di ketahui sebelum saya membuat analisis kritis dengan teori resipiositas. Perkawinan "Normal"(cara yang dianjurkan). Perkawinan dan harga seorang perempuan Perkawinan bagi orang sumba, khususnya di suku wewena sumba barat daya,dalam artiter tentu bisa saya katakan sebagai suatu upaya resiprositas antar suku,antara si



Block size     :  1024
Input ids      :  [913, 10606, 38372, 1620, 263, 5999, 461, 659, 38025, 10371, 58497, 1853, 7413, 67721, 8656, 37560, 25, 24200, 48228, 84, 17922, 77753, 6255, 11, 2953, 38072, 391, 8656, 595, 466, 28047, 2185, 24200, 16584, 72, 13, 422, 2780, 48228, 84, 17922, 281, 4204, 1154, 5676, 38072, 391, 8656, 305, 52298, 11, 1346, 524, 9101, 32720, 14225, 19593, 1853, 293, 14077, 33701, 372, 1466, 77753, 6255, 13, 47769, 14225, 37560, 817, 71, 3473, 276, 595, 4648, 77753, 6255, 10371, 10207, 65, 306, 3101, 67720, 38788, 817, 3262, 10386, 11, 58622, 38306, 524, 50977, 392, 26014, 58622, 9642, 300, 24074, 48228, 84, 17922, 77753, 6255, 32670, 4462, 22222, 84938, 6368, 524, 595, 466, 28047, 2185, 9101, 5931, 269, 16584, 72, 11, 58622, 259, 9817, 50977, 747, 585, 16225, 85140, 1656, 285, 13, 56987, 391, 79042, 4284, 37560, 90093, 391, 9243, 3101, 28337, 64, 320, 74, 6535, 44256, 93924, 568, 1649, 74977, 817, 2712, 32283, 11, 90093, 391, 11806, 52091, 332, 22891, 37560, 924

' suami atau calon istri .Dan yang harus di persiapkan adalah: dari keluarga wanita, menyiapkan kain sarung dari babi. Dari keluarga pria ,menyiapkan hewan, parang dan mamoli untuk di bawa kerumah wanita. Mamoli adalah perhiasan khas wanita yang berbentuk seperti vagina perempuan, sebagai lambang kesuburan sebagai balasanya keluarga wanita akan memberikan beberapa pasang kain sarung dan seekor babi, sebagai tanda kesepakatan jumlah belis. Tahap kedua adalah tahap masuk minta (kette katonga).Setelah perkenalan, tahap selanjutnya adalah masuk minta atau ikat adat (kette nakatonga), yang berarti mengikat atau meresmikan hubungan antara pria dan wanita serta bermakna mengikat dan melarang, karena telah terjadi kesepakatan dari kedua keluarga calon pengantin laki-laki dan perempuan yang di satukan melalui pengikatan janji. Hal ini bermakna bahwa gadis telah di pinang (Diikat) dan melarang orang lain melamarnya atau meminangnya lagi. Baik laki-laki maupun perempuan diikat dan dilarang untuk 



Block size     :  1024
Input ids      :  [14435, 300, 1103, 44570, 42369, 72742, 29594, 569, 74, 14435, 300, 12417, 6032, 277, 7777, 70, 585, 595, 11619, 16470, 276, 1538, 1420, 84, 10890, 3843, 300, 9101, 10414, 44117, 6871, 276, 24200, 730, 14435, 300, 12417, 13, 425, 17106, 42369, 10152, 6255, 12690, 49748, 63952, 2185, 24200, 21511, 85190, 20414, 45885, 370, 2185, 1853, 40013, 13434, 730, 14435, 300, 12417, 1853, 70002, 40, 151645, 35, 16823, 1962, 46453, 454, 276, 511, 75907, 2832, 2780, 11, 54211, 81237, 36686, 3101, 26014, 19593, 10207, 45497, 1962, 712, 9835, 276, 11, 15602, 2143, 24200, 40418, 351, 18633, 11, 66865, 11, 595, 2367, 9198, 276, 11, 7557, 10215, 11, 305, 65401, 36686, 8858, 18633, 274, 1735, 13, 15098, 3101, 44776, 90543, 75043, 36686, 3101, 26014, 10371, 44117, 294, 2015, 7523, 51316, 77497, 49607, 1853, 49016, 685, 11, 41068, 332, 12540, 8656, 68886, 36686, 3101, 26014, 10371, 1853, 585, 1963, 72327, 2590, 300, 3914, 13, 20957, 839, 72, 76983, 827, 10352, 258,

'ompasianacom juga Tiktok @kompasiana biar nggak ketinggalan event seru komunitas dan tips dapat cuan dari Kompasiana. Baca juga cerita inspiratif langsung dari smartphone kamu dengan bergabung di WhatsApp Channel Kompasiana di SINI<|im_end|>Dalam kehidupan sehari-hari, kita melakukan pengukuran untuk berbagai keperluan, mulai dari perdagangan, industri, kesehatan, transportasi, hingga pengembangan sains. Untuk mendapatkan hasil pengukuran yang dapat diterima oleh semua orang di dunia, dibutuhkan sistem pengukuran yang diakui secara internasional. Metrologi merupakan disiplin ilmu yang mempelajari cara-cara pengukuran, kalibrasi dan akurasi di bidang industri, ilmu pengetahuan dan teknologi. Kegiatan metrologi yang berbasis pada satuan ukuran dalam Sistem Internasional Satuan atau The International System of Units (SI). Prabowo-Sandi Tolak Hasil Pemilu 2019 dan Minta Pendukung Jaga Ketertiban Saat Demokrat dan PAN Dikabarkan Merapat ke Kubu Jokowi, Cak Imin: Saya Kira Cukup SI merupaka



Block size     :  1024
Input ids      :  [23604, 484, 74286, 1962, 17102, 356, 573, 258, 524, 11, 27853, 2068, 31811, 4350, 84, 2780, 220, 17, 15, 15, 22, 11426, 2185, 23604, 484, 1466, 1962, 259, 22051, 276, 4389, 300, 26692, 4883, 66214, 69830, 1853, 4148, 562, 11, 619, 14077, 4716, 266, 59469, 330, 67, 30503, 672, 266, 404, 8656, 32670, 294, 46447, 12589, 1, 259, 22051, 276, 1853, 356, 573, 258, 524, 13, 28634, 55487, 21447, 2267, 3651, 461, 21447, 2267, 10371, 27281, 71136, 6051, 10215, 57523, 48228, 50998, 276, 35070, 7564, 269, 23968, 151645, 43, 71739, 467, 471, 40929, 20523, 359, 9984, 905, 11, 434, 388, 1103, 355, 467, 23557, 4984, 3256, 1861, 68578, 20851, 11, 619, 11907, 2992, 32, 481, 3651, 461, 44776, 278, 10606, 294, 36960, 276, 993, 24074, 281, 1466, 309, 23165, 321, 2577, 1466, 1853, 220, 17, 18, 62162, 269, 20165, 321, 2577, 266, 360, 10240, 258, 320, 65651, 8, 1853, 11806, 324, 12540, 23968, 13, 73676, 344, 19858, 300, 3651, 461, 11960, 23166, 3651, 422, 19048, 2340

' dipindahkan ke LP Cipinang, tetapi pada Januari 2007 Landung dipindah ke tahanan Markas Komando Brigade Mobil di Depok, Jawa Barat karena "dikhawatirkan akan dianiaya" tahanan di Cipinang. Referensi Tokoh Polri Tokoh yang tidak memiliki informasi tahun kelahiran Koruptor Indonesia<|im_end|>Laporan Wartawan Tribunnews.com, Fersianus Waku TRIBUNNEWS.COM, JAKARTA - Polri mendalami dugaan adanya paham khilafah di 23 Kantor Khilafatul Muslimin (KM) di seluruh Indonesia. Kadiv Humas Polri Irjen Pol Dedi Prasetyo mengatakan, pihaknya bekerja dengan Polres dan Polda untuk mendalami dugaan tersebut. "Semuanya didalami. Ada 23 kantor wilayah kelompok KM itu lagi didalami oleh teman-teman baik dari Polres, Polda, termasuk back up dari Mabes Polri Densus 88," kata Dedi di Mako Brimob, Kelapa Dua, Depok, Jawa Barat, Sabtu (11/6/2022). Dedi menuturkan, termasuk dari Pulau Kalimantan dan Sulawesi juga didalami serta dilakukan maping. Dari hasil maping, kata dia, dilakukan pendalaman sejauh mana ket



Block size     :  1024
Input ids      :  [1982, 92046, 29792, 10207, 45497, 29180, 3101, 13, 8907, 88994, 37560, 84938, 683, 2267, 4443, 372, 25, 2573, 67721, 29180, 3101, 20847, 44547, 16806, 44117, 71136, 40269, 585, 10371, 10207, 2721, 64, 1455, 13830, 31811, 31954, 6850, 11, 1982, 70, 517, 2185, 31811, 24245, 21226, 11, 10651, 10215, 11, 9101, 20153, 21167, 9908, 300, 61087, 13, 13550, 1466, 10606, 92933, 3065, 370, 20847, 44547, 37560, 8688, 83502, 20189, 287, 29792, 705, 12589, 281, 763, 70, 22051, 9101, 5750, 18633, 71098, 13, 2823, 74402, 66792, 269, 10371, 44117, 10207, 39537, 1332, 52813, 1982, 31245, 391, 1982, 62086, 61087, 20847, 44547, 3196, 5059, 50376, 25, 8031, 54425, 19048, 19262, 42227, 33755, 74812, 2353, 300, 10371, 42227, 8212, 11, 10371, 44117, 76234, 49607, 4704, 15428, 49935, 10207, 5559, 19593, 81237, 259, 484, 19262, 4184, 49748, 14197, 6595, 18116, 332, 16540, 325, 12133, 55487, 63952, 2185, 13, 23663, 23557, 20847, 44547, 1420, 287, 4704, 15428, 1982, 75,

' terjadi dalam berbagai bentuk. Berikut adalah beberapa contoh umum: Setiap bentuk cyber bullying ini dapat memiliki dampak yang berbeda-beda pada korban, tergantung pada intensitas, durasi, dan konteks situasinya. Memahami penyebab cyber bullying adalah langkah penting dalam upaya pencegahan dan penanganannya. Beberapa faktor yang dapat berkontribusi terhadap terjadinya cyber bullying antara lain: Internet menyediakan tingkat anonimitas yang tinggi, yang dapat membuat orang merasa lebih berani untuk melakukan tindakan negatif tanpa takut konsekuensi langsung. Pelaku cyber bullying sering merasa terlindungi oleh layar komputer atau smartphone mereka, sehingga mereka cenderung melakukan tindakan yang mungkin tidak akan mereka lakukan dalam interaksi tatap muka. Interaksi online dapat mengurangi rasa empati karena pelaku tidak dapat melihat reaksi langsung dari korban. Hal ini dapat menyebabkan pelaku kurang memahami dampak dari tindakan mereka terhadap perasaan dan kesejahteraan korban

# Inisialisasi

# QLoRA Config

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

## Load Base Model

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=quant_config,
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
print(base_model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06

## Konfigurasi Lora

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 21,823,488 || all params: 8,212,558,848 || trainable%: 0.2657


## Data Collator

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Metrics Callback

## Callback untuk menyimpan semua metrik (train/eval, loss/perplexity)

In [ ]:
import math
import wandb
from transformers import TrainerCallback

class WandBPerplexityCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.metrics = {
            'train_loss': [],
            'train_perplexity': [],
            'eval_loss': [],
            'eval_perplexity': []
        }

    def on_log(self, args, state, control, logs=None, **kwargs):
        """Dipanggil setiap 'logging_steps' (saat training)"""
        if logs and 'loss' in logs:
            step = state.global_step
            loss = logs['loss']

            try:
                ppl = math.exp(loss)
            except OverflowError:
                ppl = float('inf')

            wandb.log({"train/perplexity": ppl})

            self.metrics['train_loss'].append((step, loss))
            self.metrics['train_perplexity'].append((step, ppl))

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Dipanggil setiap 'eval_steps' (saat validasi)"""
        if metrics and 'eval_loss' in metrics:
            step = state.global_step
            loss = metrics['eval_loss']

            try:
                ppl = math.exp(loss)
            except OverflowError:
                ppl = float('inf')

            wandb.log({"eval/perplexity": ppl})

            self.metrics['eval_loss'].append((step, loss))
            self.metrics['eval_perplexity'].append((step, ppl))

metrics_callback = WandBPerplexityCallback()

# Setup Up Monitoring WANDB

In [ ]:
import wandb
from dotenv import load_dotenv

env_path = '/content/drive/MyDrive/secrets/.env'
load_dotenv(env_path)
wandb_key = os.getenv('WANDB_API_KEY')
wandb.login(key=wandb_key)
wandb.init(project="CPT Tim 1", entity="Tim-1", name="Qwen3-8B-All_Data_200M Token")

wandb: Currently logged in as: taqiyudinmiftah-02 (Tim-1) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Setup untuk Trainer

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/GColab_AITFIndonesia/Qwen-Checkpoint"
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,  # Training skala besar  : 4
    per_device_eval_batch_size=4,   # Training skala besar  : 4
    gradient_accumulation_steps=16,
    eval_strategy="steps",
    eval_steps=700,
    logging_steps=50,
    save_steps=100,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=False,
    bf16=True,
    report_to="wandb",
    save_total_limit=None,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[metrics_callback],
)

/tmp/ipython-input-3168915875.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Pengecekan Checkpoint dan Memulai Continued Pre-Training

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

219

In [ ]:
from transformers.trainer_utils import get_last_checkpoint
import os

last_checkpoint = None
if os.path.isdir(OUTPUT_DIR):
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint:
    print(f"Checkpoint ditemukan di: {last_checkpoint}")
    print("Melanjutkan training dari langkah terakhir...")
else:
    print("Tidak ada checkpoint valid ditemukan. Memulai training dari awal...")

train_result = trainer.train(resume_from_checkpoint=last_checkpoint)

trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Checkpoint ditemukan di: /content/drive/MyDrive/GColab_AITFIndonesia/Qwen-Checkpoint/checkpoint-600
Melanjutkan training dari langkah terakhir...


	eval_steps: 700 (from args) != 200 (from trainer_state.json)


Step,Training Loss,Validation Loss
800,1.286000,1.293398
1000,1.282200,1.287120
1200,1.279300,1.281567
1400,1.281900,1.277482
1600,1.254900,1.273499
1800,1.247000,1.270357
2000,1.263500,1.267953
2200,1.267700,1.266186


# Evaluate

In [ ]:
trainer.evaluate()

In [ ]:
import math

eval_loss = trainer.evaluate()['eval_loss']
perplexity = math.exp(eval_loss)

print(f"Perplexity: {perplexity}")

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

train_loss_steps = [data[0] for data in metrics_callback.metrics['train_loss']]
train_loss_values = [data[1] for data in metrics_callback.metrics['train_loss']]

eval_loss_steps = [data[0] for data in metrics_callback.metrics['eval_loss']]
eval_loss_values = [data[1] for data in metrics_callback.metrics['eval_loss']]

ax1.plot(train_loss_steps, train_loss_values, label='Training Loss', alpha=0.7)
ax1.plot(eval_loss_steps, eval_loss_values, label='Validation Loss', alpha=0.7)
ax1.set_xlabel('Steps')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True)

train_ppl_steps = [data[0] for data in metrics_callback.metrics['train_perplexity']]
train_ppl_values = [data[1] for data in metrics_callback.metrics['train_perplexity']]

eval_ppl_steps = [data[0] for data in metrics_callback.metrics['eval_perplexity']]
eval_ppl_values = [data[1] for data in metrics_callback.metrics['eval_perplexity']]

ax2.plot(train_ppl_steps, train_ppl_values, label='Training Perplexity', alpha=0.7)
ax2.plot(eval_ppl_steps, eval_ppl_values, label='Validation Perplexity', alpha=0.7)
ax2.set_xlabel('Steps')
ax2.set_ylabel('Perplexity')
ax2.set_title('Training & Validation Perplexity')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Save Model

In [ ]:
print("Pelatihan selesai. Menyimpan model final.")
trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final_model"))